In [3]:
cd /home

/home


# Astromer I vs Astromer II

In [4]:
import matplotlib.pyplot as plt 
import tensorflow as tf 
import pandas as pd
import numpy as np
import glob
import os

from sklearn.metrics import r2_score, mean_squared_error
from src.training.utils import train
from src.utils import get_metrics
from matplotlib import gridspec
from src.data import load_data

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})
# for Palatino and other serif fonts use:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})

%load_ext autoreload
%autoreload 2

In [ ]:
folder_2 = './presentation/experiments/astromer_2/results/paper_nsp/2023-09-05_15-36-13/classification/alcock/fold_0/alcock_20/metrics.toml'